## Setup


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import tensorflow as tf
from tensorflow.keras import models
from datagen import DataGenerator # Custom class import

Set global variables.

Paths should be formatted _exactly_ as shown.


In [ ]:
SAVE_PATH = "/path/to/save/"
LOAD_PATH = "/path/to/load/"

PARAMS = {"batch_size": 8,
          "views": 3,
          "planes": 500,
          "cells": 500,
          "n_channels": 3}

TEST_FILES = [file[:-3] for file in glob("/path/to/batch/images.gz")]
TEST_GENERATOR = DataGenerator(TEST_FILES, **PARAMS)

Accelerate performance.


In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

## Run Tests


Reload the trained model.


In [ ]:
model = tf.keras.models.load_model(LOAD_PATH)
model.summary()

First, directly evaluate the test loss; then, compare predicted values with actual values for the energy.

Labels are log-scaled to allow for better visualization.


In [ ]:
test_loss = model.evaluate(TEST_GENERATOR, verbose=4)
print(test_loss)

test_pred = np.log(model.predict(TEST_GENERATOR).flatten())
test_labels = []
for file in TEST_GENERATOR.files:
    info = open(file + ".info", "rb").readlines()
    nuenergy = float(info[1])
    test_labels.append(nuenergy)
test_labels = np.log(np.array(test_labels))

Visualize the comparisons using matplotlib.

Block 1: scatter plot.
Block 2: 2D histogram.


In [ ]:
plt.axes(aspect="equal")
plt.xlim([-2,5])
plt.ylim([-2,5])
plt.axline((0, 0), slope=1, color="black", lw=0.5)
plt.scatter(test_labels, test_pred, s=1.5)
plt.xlabel("Actual [ln(GeV)]")
plt.ylabel("Predicted [ln(GeV)]")
plt.savefig(SAVE_PATH+"test_loss.png")

In [ ]:
plt.clf()

plt.axes(aspect="equal")
plt.hist2d(test_labels, test_pred,
           bins=35, range=[[-2,5],[-2,5]], cmin=1, cmap="Blues");
plt.xlabel("Actual [ln(GeV)]")
plt.ylabel("Predicted [ln(GeV)]")
plt.savefig(SAVE_PATH+"test_loss_hist.png")